In [ ]:
import pandas as pd
import numpy as np
from joblib import Parallel, delayed
from tqdm import tqdm

def cross_correlation(series1, series2, lag=0):
    if lag < 0:
        raise ValueError("Lag must be non-negative")
    # Align the series based on lag
    if lag == 0:
        s1 = series1
        s2 = series2
    else:
        s1 = series1[lag:]
        s2 = series2[:-lag]
    # Drop missing values from both series
    valid = s1.notna() & s2.notna()
    s1 = s1[valid]
    s2 = s2[valid]
    
    if len(s1) < 2 or len(s2) < 2:
        return np.nan
    
    # Convert to numpy arrays and compute standard deviations
    x = s1.to_numpy()
    y = s2.to_numpy()
    std_x = np.std(x, ddof=1)
    std_y = np.std(y, ddof=1)
    
    if std_x == 0 or std_y == 0:
        return np.nan
    
    # Compute covariance with Bessel's correction
    cov = np.cov(x, y, ddof=1)[0, 1]
    return cov / (std_x * std_y)

def process_trajectory(traj, group, other_features, lags):
    """
    Process a single patient trajectory. For each feature and each lag,
    compute the cross-correlation with 'o:SOFA' after dropping missing values.
    """
    local_results = {feature: {lag: [] for lag in lags} for feature in other_features}
    group = group.sort_values('step')
    sofa_series = group['o:SOFA']
    
    for feature in other_features:
        feature_series = group[feature]
        # Compute a valid mask once for this feature
        valid = sofa_series.notna() & feature_series.notna()
        sofa_valid = sofa_series[valid]
        feature_valid = feature_series[valid]
        
        for lag in lags:
            # Only compute if enough data points exist after alignment
            if len(sofa_valid) > lag:
                corr_value = cross_correlation(sofa_valid, feature_valid, lag)
                if not np.isnan(corr_value):
                    local_results[feature][lag].append(corr_value)
    return local_results

def merge_results(results, other_features, lags):
    """
    Merge the results (lists of correlation values) from each trajectory.
    """
    merged = {feature: {lag: [] for lag in lags} for feature in other_features}
    for res in results:
        for feature in other_features:
            for lag in lags:
                merged[feature][lag].extend(res[feature][lag])
    return merged

def main():
    # Update the file path as necessary
    csv_file = "./sepsis_final_data_RAW_withTimes_continuous.csv"
    df = pd.read_csv(csv_file)
    df = df.sort_values(['traj', 'step'])
    
    # Identify observation columns (those starting with "o")
    obs_cols = [col for col in df.columns if col.startswith('o')]
    if 'o:SOFA' not in obs_cols:
        raise ValueError("Column 'o:SOFA' not found in dataset")
    
    # List of features to compare with o:SOFA (excluding itself)
    other_features = [col for col in obs_cols if col != 'o:SOFA']
    
    # Remove specific columns that are not required
    for col in ["o:max_dose_vaso", "o:input_4hourly", "o:age", "o:gender", "o:re_admission"]:
        if col in other_features:
            other_features.remove(col)
    
    # Define the lags to compute
    lags = [0, 1, 2, 3]
    
    # Group data by patient trajectory
    grouped = list(df.groupby('traj'))
    
    # Process each trajectory in parallel
    results = Parallel(n_jobs=-1)(
        delayed(process_trajectory)(traj, group, other_features, lags)
        for traj, group in tqdm(grouped, desc="Processing trajectories")
    )
    
    # Merge results from all trajectories
    merged_results = merge_results(results, other_features, lags)
    
    # Average the correlations for each feature and lag across patients
    avg_corr = {feature: {} for feature in other_features}
    for feature in other_features:
        for lag in lags:
            if len(merged_results[feature][lag]) > 0:
                avg_corr[feature][lag] = np.mean(merged_results[feature][lag])
            else:
                avg_corr[feature][lag] = np.nan
    
    # Convert the results into a DataFrame for easier viewing
    avg_corr_df = pd.DataFrame(avg_corr).T
    avg_corr_df = avg_corr_df[lags]  # Ensure columns are in lag order
    print(avg_corr_df)
    
    return avg_corr_df

In [2]:
if __name__ == "__main__":
    avg_corr_df = main()

Processing trajectories: 100%|██████████| 18913/18913 [03:20<00:00, 94.37it/s] 


                            0         1         2         3
o:mechvent           0.176696  0.264492  0.274567  0.257572
o:Weight_kg         -0.005629 -0.023109 -0.026958 -0.021322
o:GCS               -0.444028 -0.437736 -0.435731 -0.428769
o:HR                 0.007500 -0.014189 -0.025463 -0.032236
o:SysBP             -0.111810 -0.130255 -0.141662 -0.141074
o:MeanBP            -0.138606 -0.155146 -0.167490 -0.171930
o:DiaBP             -0.115162 -0.125050 -0.135155 -0.131484
o:RR                -0.012927 -0.039910 -0.046301 -0.042172
o:Temp_C             0.016114  0.009205  0.009203  0.012268
o:FiO2_1             0.204245  0.216806  0.207921  0.200784
o:Potassium          0.031623  0.021023  0.024423  0.020321
o:Sodium            -0.034941 -0.031261 -0.029088 -0.015920
o:Chloride          -0.060965 -0.016401 -0.001876 -0.004177
o:Glucose            0.004725  0.022217  0.023451  0.020999
o:Magnesium          0.003279 -0.010970 -0.012699 -0.005220
o:Calcium            0.025881  0.007866 

In [8]:
selected_indices = avg_corr_df[(avg_corr_df.abs() > 0.2).any(axis=1)].index
print(selected_indices)

Index(['o:mechvent', 'o:GCS', 'o:FiO2_1', 'o:paO2', 'o:PaO2_FiO2',
       'o:Total_bili', 'o:output_4hourly'],
      dtype='object')


In [ ]:
import pandas as pd
import numpy as np
from joblib import Parallel, delayed
from tqdm import tqdm

def process_feature_pair(feature_1, feature_2, groups, lags):
    # Initialize a dictionary to store shifted arrays for each lag.
    pooled = {lag: {"x1": [], "x2": []} for lag in lags}
    
    for group in groups:
        arr1 = group[feature_1].values
        arr2 = group[feature_2].values
        n = len(group)
        
        # For lag 0, require at least 2 data points.
        if n >= 2:
            pooled[0]["x1"].append(arr1)
            pooled[0]["x2"].append(arr2)
            
        # For positive lags, slice the arrays if enough data exists.
        for lag in lags[1:]:
            if n > lag:
                pooled[lag]["x1"].append(arr1[lag:])
                pooled[lag]["x2"].append(arr2[:-lag])
    
    # Compute correlations for each lag.
    results = []
    for lag in lags:
        if not pooled[lag]["x1"]:
            corr = np.nan
        else:
            x1 = np.concatenate(pooled[lag]["x1"])
            x2 = np.concatenate(pooled[lag]["x2"])
            # Check for sufficient data and non-constant arrays.
            if len(x1) < 2 or np.std(x1) == 0 or np.std(x2) == 0:
                corr = np.nan
            else:
                corr = np.corrcoef(x1, x2)[0, 1]
        results.append({
            'feature_1': feature_1,
            'feature_2': feature_2,
            'lag': lag,
            'correlation': corr
        })
    return results

def pooled_lagged_correlation_main():
    # Load dataset
    df = pd.read_csv("./sepsis_final_data_RAW_withTimes_continuous.csv")
    
    # Ensure data are sorted by patient ('traj') and time ('step')
    df = df.sort_values(['traj', 'step'])
    
    # Identify observation columns (those starting with "o")
    obs_cols = [col for col in df.columns if col.startswith('o')]
    # Remove specific columns not needed
    cols_to_remove = ["o:max_dose_vaso", "o:input_4hourly", "o:gender", "o:re_admission", "o:age", "o:SOFA"]
    for col in cols_to_remove:
        if col in obs_cols:
            obs_cols.remove(col)
    
    # Define two sets of features.
    sel_cols_1 = ['o:mechvent', 'o:GCS', 'o:FiO2_1', 'o:paO2', 'o:PaO2_FiO2',
                  'o:Total_bili', 'o:output_4hourly']
    sel_cols_2 = [col for col in obs_cols if col not in sel_cols_1]
    
    # Define the lags to search (0 to 3).
    lags = [0, 1, 2, 3]
    
    # Cache groups: list of DataFrames for each unique 'traj', sorted by 'step'.
    groups = [group.sort_values('step') for _, group in df.groupby('traj')]
    
    # Prepare the list of feature pairs (compute each pair only once).
    feature_pairs = [(f1, f2) for f1 in sel_cols_1 for f2 in sel_cols_2]
    
    # Process feature pairs in parallel.
    all_results = Parallel(n_jobs=-1)(
        delayed(process_feature_pair)(f1, f2, groups, lags)
        for f1, f2 in tqdm(feature_pairs, desc="Processing feature pairs")
    )
    
    # Flatten the list of lists into a single list of dictionaries.
    flattened_results = [item for sublist in all_results for item in sublist]
    
    # Convert the results to a DataFrame and return.
    results_df = pd.DataFrame(flattened_results)
    return results_df

if __name__ == '__main__':
    results_df = pooled_lagged_correlation_main()
    # For demonstration, print the first few rows of the DataFrame.
    print(results_df.head())

Processing feature pairs: 100%|██████████| 238/238 [04:18<00:00,  1.09s/it]


    feature_1    feature_2  lag  correlation
0  o:mechvent  o:Weight_kg    0     0.071849
1  o:mechvent  o:Weight_kg    1     0.071639
2  o:mechvent  o:Weight_kg    2     0.070113
3  o:mechvent  o:Weight_kg    3     0.067919
4  o:mechvent         o:HR    0     0.018215


In [17]:
results_df_filter = results_df[results_df['correlation'].abs() >= 0.2]
results_df_filter

,feature_1,feature_2,lag,correlation
104,o:mechvent,o:SpO2,0,0.217191
105,o:mechvent,o:SpO2,1,0.218067
106,o:mechvent,o:SpO2,2,0.203638
128,o:mechvent,o:input_total,0,0.247055
129,o:mechvent,o:input_total,1,0.233322
130,o:mechvent,o:input_total,2,0.229206
131,o:mechvent,o:input_total,3,0.229378
948,o:output_4hourly,o:output_total,0,0.279923
949,o:output_4hourly,o:output_total,1,0.239853
950,o:output_4hourly,o:output_total,2,0.224943


In [ ]:
df = pd.read_csv("./sepsis_final_data_RAW_withTimes_continuous.csv")
ls_cols_sel = ['traj', 'step', "o:gender", "o:re_admission", "o:age", 'o:Weight_kg', 'o:mechvent', 'o:GCS', 'o:FiO2_1', 'o:paO2', 'o:PaO2_FiO2',  'o:Total_bili', 'o:output_4hourly', 'o:output_total', 'o:input_total', 'o:SpO2', 'o:max_dose_vaso', 'o:input_4hourly', 'o:SOFA', 'r:reward']
df = df[ls_cols_sel]
df.rename(columns={'o:max_dose_vaso': 'a:max_dose_vaso'}, inplace=True)
df.rename(columns={'o:input_4hourly': 'a:input_4hourly'}, inplace=True)
df.rename(columns={'o:SOFA': 'r:SOFA'}, inplace=True)
df['r:SOFA'] = -df['r:SOFA'].astype(np.float32)
df.rename(columns={'o:Weight_kg': 'co:Weight_kg'}, inplace=True)
df.to_csv("sepsis_final_RAW_continuous_13_weights.csv", index=False)
df = df.drop(columns=['co:Weight_kg'])
df.to_csv("sepsis_final_RAW_continuous_13.csv", index=False)